# **LLMs Locales en la Nube: Ollama + RAG con LangChain**

**Objetivo de la clase:**
Aprender a levantar un servidor de LLMs en Google Colab (GPU gratuita),
exponerlo al exterior con ngrok, y construir un sistema RAG (Retrieval-Augmented
Generation) con LangChain que responda preguntas sobre documentos propios.

## ¿Qué es RAG y por qué importa?

Un LLM "de base" solo conoce lo que vio durante su entrenamiento.
RAG resuelve esto en 3 pasos:

  1. 📄 **Indexar** tus documentos (convertirlos en vectores)
  2. 🔍 **Recuperar** los fragmentos más relevantes para cada pregunta
  3. 💬 **Generar** una respuesta usando esos fragmentos como contexto

```
Pregunta ──► Recuperador ──► Fragmentos relevantes ──► LLM ──► Respuesta
                  ▲
            Base vectorial
            (tus documentos)
```

## Requisitos previos
- Cuenta Google con acceso a Colab
- Cuenta ngrok gratuita: https://dashboard.ngrok.com/signup
- Auth Token de ngrok: https://dashboard.ngrok.com/get-started/your-authtoken

⚠️ Antes de empezar: **Runtime → Change runtime type → T4 GPU**

In [1]:
# ===========================================================================
# PASO 0: Verificar GPU
# ===========================================================================
# Siempre verificamos primero que tenemos GPU disponible.
# Sin GPU, los modelos tardan 10-20x más → clase inviable.

import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if result.returncode != 0:
    raise RuntimeError(
        "❌ No se detectó GPU.\n"
        "Ve a Runtime → Change runtime type → T4 GPU y vuelve a ejecutar."
    )

print("✅ GPU disponible:")
print(result.stdout.split('\n')[8])  # Línea con el modelo de GPU

✅ GPU disponible:
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |


In [2]:

# ===========================================================================
# PASO 1: Instalar Ollama
# ===========================================================================
# Ollama es un servidor que gestiona modelos LLM localmente.
# Expone una API REST en el puerto 11434, compatible con la API de OpenAI.
#
# Dependencias del sistema:
#   - zstd: para descomprimir el binario de Ollama
#   - pciutils: para que Ollama detecte la GPU (¡sin esto corre en CPU!)

import os
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

print("📦 Instalando dependencias del sistema...")
!sudo apt-get update -qq 2>/dev/null
!sudo apt-get install -y -qq zstd pciutils 2>/dev/null

print("\n📥 Instalando Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh 2>&1 | grep -v 'systemd'

!ollama --version

📦 Instalando dependencias del sistema...
Selecting previously unselected package pci.ids.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1ubuntu0.1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Selecting previously unselected package libpci3:amd64.
Preparing to unpack .../libpci3_1%3a3.7.0-6_amd64.deb ...
Unpacking libpci3:amd64 (1:3.7.0-6) ...
Selecting previously unselected package pciutils.
Preparing to unpack .../pciutils_1%3a3.7.0-6_amd64.deb ...
Unpacking pciutils (1:3.7.0-6) ...
Selecting previously unselected package zstd.
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Setting up libpci3:amd64 (1:3.7.0-6) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Setting up pciutils (1:3.7.0-6) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/s

In [ ]:
# ===========================================================================
# PASO 2: Instalar librerías Python
# ===========================================================================
# - pyngrok: cliente Python para el túnel ngrok
# - ollama: cliente oficial para la API de Ollama
# - langchain + langchain-ollama: framework de orquestación de LLMs
# - langchain-community: integraciones de terceros (vectorstores, loaders, etc.)
# - faiss-cpu: librería de búsqueda vectorial eficiente (de Meta)
#   Usamos la versión CPU porque FAISS corre en RAM, no en VRAM

print("📦 Instalando librerías Python...")
!pip install -qq pyngrok ollama
!pip install -qq langchain langchain-ollama langchain-community
!pip install -qq faiss-cpu

In [ ]:
# ===========================================================================
# PASO 3: Configurar ngrok
# ===========================================================================
# ngrok crea un túnel HTTPS entre internet y un puerto local de Colab.
# Necesitamos esto porque Colab no tiene IP pública directa.
#
# Configura tu token en Colab Secrets (icono 🔑 en el panel izquierdo):
#   Nombre: NGROK_AUTHTOKEN
#   Valor: tu token de https://dashboard.ngrok.com/get-started/your-authtoken

from google.colab import userdata

try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    print(f"✅ Token cargado: {NGROK_AUTHTOKEN[:8]}...{NGROK_AUTHTOKEN[-4:]}")
except Exception:
    raise ValueError(
        "❌ Token de ngrok no encontrado.\n"
        "Añádelo en Colab Secrets (🔑) con el nombre NGROK_AUTHTOKEN."
    )

In [ ]:
# ===========================================================================
# PASO 4: Iniciar el servidor Ollama
# ===========================================================================
# Lanzamos Ollama como proceso en background.
# OLLAMA_HOST=0.0.0.0 es necesario para que ngrok pueda alcanzarlo.

import subprocess, time, requests

os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'

print("🚀 Iniciando servidor Ollama...")
ollama_proc = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)

# Verificar que el servidor responde
for intento in range(3):
    try:
        r = requests.get('http://localhost:11434', timeout=10)
        print(f"✅ Servidor activo: {r.text.strip()}")
        break
    except requests.ConnectionError:
        print(f"   Intento {intento + 1}/3... esperando")
        time.sleep(5)
else:
    raise RuntimeError("❌ El servidor Ollama no arrancó correctamente.")
